In [ ]:
"""
Created on Fri Apr 8 10:29 2022

Prepare reference metrics in NEMO for tuning and evaluation

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
from basal_melt_param.constants import *
import matplotlib as mpl
import glob

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
nemo_run = 'isfru94' # 'EPM026','EPM031', 'EPM034'
if nemo_run in ['ctrl94']:
    yy_start = 1982
    yy_end = 2013
elif nemo_run in ['isf94','isfru94']:
    yy_start = 2014
    yy_end = 2100

READ IN DATA

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_TUNING/nemo_5km_'+nemo_run+'/'
outputpath = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/'

In [ ]:
file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

file_conc = xr.open_mfdataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)#.chunk(chunks={'x': chunk_size, 'y': chunk_size})

In [ ]:
isf_mask = (file_isf['ISF_mask'] == file_isf.Nisf)

In [ ]:
xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)

In [ ]:
melt_files = list(sorted(glob.glob(inputpath_data+'cavity_melt_*_Ant_stereo.nc')))
ds_melt  = xr.open_mfdataset(melt_files, concat_dim='time_counter', combine='nested')#, chunks={'x': chunksize, 'y': chunksize})
ds_melt = ds_melt.rename({'time_counter': 'time'})
ds_melt = ds_melt.assign_coords(time=np.arange(yy_start, yy_end+1))
ds_melt_cutted = uf.cut_domain_stereo(ds_melt, map_lim, map_lim)

In [ ]:
ds_melt_cutted.to_netcdf(inputpath_data+'cavity_melt_'+str(yy_start)+'-'+str(yy_end)+'_Ant_stereo.nc')

2D m ice per y

In [ ]:
ds_melt_cutted = xr.open_mfdataset(inputpath_data+'cavity_melt_'+str(yy_start)+'-'+str(yy_end)+'_Ant_stereo.nc')

In [ ]:
melt_m_ice_per_y_whole_cell = (ds_melt_cutted['melt_cavity']*yearinsec/rho_i)
melt_m_ice_per_y = melt_m_ice_per_y_whole_cell/file_conc_cut['isfdraft_conc']

In [ ]:
melt_m_ice_per_y.to_dataset(name='melt_m_ice_per_y').to_netcdf(outputpath_melt+'melt_rates_2D_NEMO.nc')

1D 

In [ ]:
chunked_mask = isf_mask.chunk({'x': 500, 'y': 500})
file_isf_conc = file_conc['isfdraft_conc'].where(chunked_mask)
melt_m_ice_per_y_avg = melt_m_ice_per_y_whole_cell.where(chunked_mask).mean(['x','y'])

In [ ]:
NEMO_melt_rates_1D = xr.Dataset({'melt_m_ice_per_y_avg': (melt_m_ice_per_y_avg.dims, melt_m_ice_per_y_avg.data)}, 
                                coords=melt_m_ice_per_y_avg.coords)

In [ ]:
NEMO_melt_rates_1D['melt_Gt_per_y_tot'] = ds_melt_cutted['melt_cavity'].where(chunked_mask).sum(['x','y']) * grid_cell_area * yearinsec / 10**12

NEMO_melt_rates_1D['isf_area'] = (isf_mask*file_isf_conc).sum(['x','y'])*grid_cell_area

In [ ]:
#NEMO_melt_rates_1D.to_netcdf(outputpath_melt+'melt_rates_1D_NEMO.nc','w')
NEMO_melt_rates_1D.to_netcdf(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc','w')

Time mean 2D

In [ ]:
NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt+'melt_rates_2D_NEMO.nc')

In [ ]:
NEMO_melt_rates_2D_timmean = NEMO_melt_rates_2D.mean('time')
NEMO_melt_rates_2D_timmean.to_netcdf(outputpath_melt+'melt_rates_2D_NEMO_timmean.nc')

BOX 1

In [ ]:
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
# Box characteristics
#box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D.nc')
#box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D.nc')
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))

box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')

file_isf_conc = file_conc['isfdraft_conc']

# Reference 2D 
ref_melt_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO.nc')

# Reference (in m/yr)
box1_list = []
ref_box1_myr_list = []
    
for kisf in tqdm(file_isf.Nisf.values):
    isf_conc_isf = uf.choose_isf(file_isf_conc, isf_stack_mask, kisf)
    box_loc_config_stacked = uf.choose_isf(box1, isf_stack_mask, kisf)
    ref_melt_2D_box1_isf = uf.choose_isf(ref_melt_2D['melt_m_ice_per_y'], isf_stack_mask, kisf).where(np.isfinite(box_loc_config_stacked))
    ref_melt_2D_box1_isf_mean = uf.weighted_mean(ref_melt_2D_box1_isf,['mask_coord'], isf_conc_isf)
    ref_box1_myr_list.append(ref_melt_2D_box1_isf_mean)

ref_box1_myr_all = xr.concat(ref_box1_myr_list, dim='Nisf')

#ref_box1_myr_all.to_dataset(name='mean_melt_box1_myr').to_netcdf(outputpath_melt+'melt_rates_box1_NEMO.nc')
ref_box1_myr_all.to_dataset(name='mean_melt_box1_myr').to_netcdf(outputpath_melt+'melt_rates_box1_NEMO_oneFRIS.nc')